In [6]:
!pip install gensim nltk scikit-learn pandas numpy

In [ ]:
!pip install --upgrade numpy gensim
!pip install nltk scikit-learn pandas

# Reiniciar o runtime para aplicar as atualizações
import os
os._exit(00)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00


In [16]:
!pip install --upgrade nltk
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import re
import warnings
warnings.filterwarnings('ignore')
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [17]:
# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# %%
# Carregar os dados
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

# Exibir as primeiras linhas
print(df.head())

# Converter sentimentos para numérico (positivo=1, negativo=0)
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Inicializar lematizador
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [18]:
try:
    nltk.data.find('tokenizers/punkt')
    print("Recursos do NLTK carregados com sucesso!")
except LookupError:
    print("Erro ao carregar recursos. Tentando solução alternativa...")
    # Solução alternativa para ambientes com problemas de permissão
    import os
    os.makedirs('/root/nltk_data/tokenizers', exist_ok=True)
    nltk.download('punkt', download_dir='/root/nltk_data')
    nltk.data.path.append('/root/nltk_data')

Recursos do NLTK carregados com sucesso!


In [20]:
def preprocess_text(text):
    # Remover tags HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Remover caracteres especiais e números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Converter para minúsculas
    text = text.lower()
    # Tokenização
    tokens = word_tokenize(text)
    # Remover stopwords e lematização
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Aplicar pré-processamento
df['processed_review'] = df['review'].apply(preprocess_text)

# %%
# Dividir dados em treino e teste
X = df['processed_review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# %%
# 1. Bag of Words (BoW)
bow_vectorizer = CountVectorizer(max_features=5000)
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# %%
# 2. TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# %%
# 3. Word2Vec
# Tokenizar as sentenças para o Word2Vec
sentences = [word_tokenize(text) for text in X_train]

# Treinar modelo Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Função para calcular vetor médio de uma sentença
def get_sentence_vector(sentence):
    vectors = []
    for word in sentence:
        if word in word2vec_model.wv:
            vectors.append(word2vec_model.wv[word])
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Aplicar aos dados de treino e teste
X_train_word2vec = np.array([get_sentence_vector(word_tokenize(text)) for text in X_train])
X_test_word2vec = np.array([get_sentence_vector(word_tokenize(text)) for text in X_test])

In [22]:
# Dicionário para armazenar resultados
results = {}

# %%
# 1. Regressão Logística

# Com BoW
lr_bow = LogisticRegression(max_iter=1000)
lr_bow.fit(X_train_bow, y_train)
y_pred = lr_bow.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
results['Logistic Regression + BoW'] = accuracy

# Com TF-IDF
lr_tfidf = LogisticRegression(max_iter=1000)
lr_tfidf.fit(X_train_tfidf, y_train)
y_pred = lr_tfidf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
results['Logistic Regression + TF-IDF'] = accuracy

# Com Word2Vec
lr_word2vec = LogisticRegression(max_iter=1000)
lr_word2vec.fit(X_train_word2vec, y_train)
y_pred = lr_word2vec.predict(X_test_word2vec)
accuracy = accuracy_score(y_test, y_pred)
results['Logistic Regression + Word2Vec'] = accuracy

In [23]:
# 2. Random Forest

# Com BoW
rf_bow = RandomForestClassifier(n_estimators=100, random_state=42)
rf_bow.fit(X_train_bow, y_train)
y_pred = rf_bow.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
results['Random Forest + BoW'] = accuracy

# Com TF-IDF
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_tfidf.fit(X_train_tfidf, y_train)
y_pred = rf_tfidf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
results['Random Forest + TF-IDF'] = accuracy

# Com Word2Vec
rf_word2vec = RandomForestClassifier(n_estimators=100, random_state=42)
rf_word2vec.fit(X_train_word2vec, y_train)
y_pred = rf_word2vec.predict(X_test_word2vec)
accuracy = accuracy_score(y_test, y_pred)
results['Random Forest + Word2Vec'] = accuracy

# %%
# Exibir resultados
print("Resultados de Acurácia:")
for method, accuracy in results.items():
    print(f"{method}: {accuracy:.4f}")

# %%
# Criar DataFrame com resultados
results_df = pd.DataFrame(list(results.items()), columns=['Método', 'Acurácia'])
results_df = results_df.sort_values(by='Acurácia', ascending=False)

# Exibir tabela de resultados
print("\nComparação de Métodos:")
results_df

Resultados de Acurácia:
Logistic Regression + BoW: 0.8728
Logistic Regression + TF-IDF: 0.8855
Logistic Regression + Word2Vec: 0.8530
Random Forest + BoW: 0.8484
Random Forest + TF-IDF: 0.8514
Random Forest + Word2Vec: 0.8331

Comparação de Métodos:


,Método,Acurácia
1,Logistic Regression + TF-IDF,0.8855
0,Logistic Regression + BoW,0.8728
2,Logistic Regression + Word2Vec,0.8530
4,Random Forest + TF-IDF,0.8514
3,Random Forest + BoW,0.8484
5,Random Forest + Word2Vec,0.8331


In [24]:
# Preencher conclusões com os valores reais
melhor_metodo = results_df.iloc[0]['Método']
melhor_acuracia = results_df.iloc[0]['Acurácia']

print(f"\nConclusão Final:")
print(f"O método que obteve a melhor acurácia foi: {melhor_metodo} com {melhor_acuracia:.4f}")


Conclusão Final:
O método que obteve a melhor acurácia foi: Logistic Regression + TF-IDF com 0.8855
